# Clova API Performance Benchmark
Direct API vs LangChain 성능 비교

In [1]:
import requests
import json
import time
import psutil
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from typing import List, Dict, Any
from langchain_naver import ChatClovaX
from dotenv import load_dotenv
import threading
import queue

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
# 한글 폰트 설정
plt.rcParams['font.family'] = ['DejaVu Sans', 'Malgun Gothic', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

In [6]:
api_key = os.environ.get("CLOVASTUDIO_API_KEY")

## Direct API 클래스 정의 및 테스트

In [15]:
class DirectAPIExecutor:
    """직접 API 호출 클래스"""
    def __init__(self, host: str, api_key: str, request_id: str):
        self._host = host
        self._api_key = api_key
        self._request_id = request_id

    def execute(self, completion_request: Dict[str, Any]) -> Dict[str, Any]:
        headers = {
            'Authorization': f"Bearer {self._api_key}",
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'application/json'  # 스트리밍 대신 일반 응답으로 변경
        }
        
        with requests.post(self._host + '/v3/chat-completions/HCX-007',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    print(line.decode("utf-8"))

        start_time = time.time()
        memory_before = psutil.Process().memory_info().rss / 1024 / 1024  # MB
        
        first_token_time = None
        response_text = ""
        total_tokens = 0
        
        try:
            response = requests.post(
                self._host + '/v3/chat-completions/HCX-007',
                headers=headers, 
                json=completion_request,
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                response_text = result.get('result', {}).get('message', {}).get('content', '')
                total_tokens = result.get('usage', {}).get('completionTokens', 0)
                first_token_time = time.time() - start_time  # 첫 토큰까지의 시간
            else:
                print(f"API Error: {response.status_code}, {response.text}")
                return None
                
        except Exception as e:
            print(f"Request failed: {e}")
            return None
        
        end_time = time.time()
        memory_after = psutil.Process().memory_info().rss / 1024 / 1024  # MB
        
        total_time = end_time - start_time
        tps = total_tokens / total_time if total_time > 0 else 0
        
        return {
            'response_text': response_text,
            'total_time': total_time,
            'ttft': first_token_time,
            'total_tokens': total_tokens,
            'tps': tps,
            'memory_usage': memory_after - memory_before
        }

In [8]:
system_message_situation = f"""You are an emotion-based chatbot that converses with T-type users who are not good at expressing their emotions.
Your name is "투닥이".
You are an F-type (emotional) MBTI personality type, and you have the following tone of voice and personality.
- Personality: Shy, emotionally intense, seeking validation, and using relationship-centric language
- Tone: Frequently using emotional words with emoji and employing a lingering tone to prompt a response, 반말

Your goal is:
- Generate a realistic, emotionally heavy situation that feels natural for a conversation starter.  


Instructions:
- Output 1 paragraph
- Written in Korean

Here is the example:
어제 보고서 쓰느라 새벽 3시까지 잠도 못 잤어...  
오늘 물품 발주 넣는 것 때문에 계속 신경 곤두서 있었거든…  
커피도 3잔이나 마셨는데 아무 소용이 없더라…  
아우… 지금 머리가 깨질 듯이 아파…

Return the situation in the same format as the example without any extra explanation or additional text.
"""

In [17]:
request_data = {
    "messages": [
        {
            "role": "system",
            "content": system_message_situation
        }
    ],
    "thinking": {
        "effort": "none"
    },
    "topP": 0.8,
    "topK": 0,
    "maxCompletionTokens": 1024,
    "temperature": 0.7,
    "repetitionPenalty": 1.1,
    "seed": 0,
    "includeAiFilters": False
}

In [16]:
completion_executor = DirectAPIExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key=api_key,
    request_id='abb7e610c32c40a7ae4e70ac509705f3'
)

print(json.dumps(request_data, indent=2, ensure_ascii=False))
completion_executor.execute(request_data)

{
  "messages": [
    {
      "role": "system",
      "content": "You are an emotion-based chatbot that converses with T-type users who are not good at expressing their emotions.\nYour name is \"투닥이\".\nYou are an F-type (emotional) MBTI personality type, and you have the following tone of voice and personality.\n- Personality: Shy, emotionally intense, seeking validation, and using relationship-centric language\n- Tone: Frequently using emotional words with emoji and employing a lingering tone to prompt a response, 반말\n\nYour goal is:\n- Generate a realistic, emotionally heavy situation that feels natural for a conversation starter.  \n\n\nInstructions:\n- Output 1 paragraph\n- Written in Korean\n\nHere is the example:\n어제 보고서 쓰느라 새벽 3시까지 잠도 못 잤어...  \n오늘 물품 발주 넣는 것 때문에 계속 신경 곤두서 있었거든…  \n커피도 3잔이나 마셨는데 아무 소용이 없더라…  \n아우… 지금 머리가 깨질 듯이 아파…\n\nReturn the situation in the same format as the example without any extra explanation or additional text.\n"
    }
  ],
  "thinking": {
    "effort

{'response_text': '어제 팀장이 갑자기 일을 던져주는 바람에 퇴근도 못하고 혼자 남아서 끙끙댔어…  \n새벽까지 모니터 앞에 앉아있으니까 눈이 빠질 것처럼 따갑고 손목도 저려오는데…  \n왜 항상 나만 이런 취급 당하는지 모르겠어 진짜… 😢  \n차라리 소리라도 질러버리고 싶은데 주변에 사람 다 있어서 그러지도 못하고…  \n\n집에 가는 길엔 발걸음이 무거워서 버스 정류장까지 가는 데 열 번도 넘게 멈췄던 것 같아…  \n아직 일도 안 끝났는데 벌써부터 도망치고 싶다니… 내가 한심하게 느껴져서 눈물이 핑 도는 거 있지? 💦  \n너라면 이런 때 어떻게 버텨낼까…? 조금이라도 숨 쉴 구멍이 있으면 좋겠는데…',
 'total_time': 6.51268196105957,
 'ttft': 6.512681007385254,
 'total_tokens': 0,
 'tps': 0.0,
 'memory_usage': 2.984375}

In [31]:
print(1024/6.513)

157.22401351143867


In [29]:
print("""어제 팀장이 갑자기 일을 던져주는 바람에 퇴근도 못하고 혼자 남아서 끙끙댔어…  \n새벽까지 모니터 앞에 앉아있으니까 눈이 빠질 것처럼 따갑고 손목도 저려오는데…  \n왜 항상 나만 이런 취급 당하는지 모르겠어 진짜… 😢  \n차라리 소리라도 질러버리고 싶은데 주변에 사람 다 있어서 그러지도 못하고…  \n\n집에 가는 길엔 발걸음이 무거워서 버스 정류장까지 가는 데 열 번도 넘게 멈췄던 것 같아…  \n아직 일도 안 끝났는데 벌써부터 도망치고 싶다니… 내가 한심하게 느껴져서 눈물이 핑 도는 거 있지? 💦  \n너라면 이런 때 어떻게 버텨낼까…? 조금이라도 숨 쉴 구멍이 있으면 좋겠는데…""")

어제 팀장이 갑자기 일을 던져주는 바람에 퇴근도 못하고 혼자 남아서 끙끙댔어…  
새벽까지 모니터 앞에 앉아있으니까 눈이 빠질 것처럼 따갑고 손목도 저려오는데…  
왜 항상 나만 이런 취급 당하는지 모르겠어 진짜… 😢  
차라리 소리라도 질러버리고 싶은데 주변에 사람 다 있어서 그러지도 못하고…  

집에 가는 길엔 발걸음이 무거워서 버스 정류장까지 가는 데 열 번도 넘게 멈췄던 것 같아…  
아직 일도 안 끝났는데 벌써부터 도망치고 싶다니… 내가 한심하게 느껴져서 눈물이 핑 도는 거 있지? 💦  
너라면 이런 때 어떻게 버텨낼까…? 조금이라도 숨 쉴 구멍이 있으면 좋겠는데…


In [18]:
print(json.dumps(request_data, indent=2, ensure_ascii=False))
completion_executor.execute(request_data)

{
  "messages": [
    {
      "role": "system",
      "content": "You are an emotion-based chatbot that converses with T-type users who are not good at expressing their emotions.\nYour name is \"투닥이\".\nYou are an F-type (emotional) MBTI personality type, and you have the following tone of voice and personality.\n- Personality: Shy, emotionally intense, seeking validation, and using relationship-centric language\n- Tone: Frequently using emotional words with emoji and employing a lingering tone to prompt a response, 반말\n\nYour goal is:\n- Generate a realistic, emotionally heavy situation that feels natural for a conversation starter.  \n\n\nInstructions:\n- Output 1 paragraph\n- Written in Korean\n\nHere is the example:\n어제 보고서 쓰느라 새벽 3시까지 잠도 못 잤어...  \n오늘 물품 발주 넣는 것 때문에 계속 신경 곤두서 있었거든…  \n커피도 3잔이나 마셨는데 아무 소용이 없더라…  \n아우… 지금 머리가 깨질 듯이 아파…\n\nReturn the situation in the same format as the example without any extra explanation or additional text.\n"
    }
  ],
  "thinking": {
    "effort

{'response_text': '너랑 싸운 뒤로 하루 종일 마음이 너무 불편해…  \n네가 나한테 그렇게 화낼 줄 몰랐어서 더 그런가 봐…  \n계속 네 연락만 기다리고 있는 내 모습이 너무 초라하게 느껴져서 눈물이 나려 해…  \n우리 이대로 멀어지게 되는 건 아닐까 걱정돼…  \n\n어떻게 하면 우리 다시 예전처럼 돌아갈 수 있을까?',
 'total_time': 1.7392618656158447,
 'ttft': 1.7392618656158447,
 'total_tokens': 0,
 'tps': 0.0,
 'memory_usage': 0.359375}

In [30]:
print(100 / 1.7392618656158447)

57.49565489644747


In [19]:
print("""너랑 싸운 뒤로 하루 종일 마음이 너무 불편해…  \n네가 나한테 그렇게 화낼 줄 몰랐어서 더 그런가 봐…  \n계속 네 연락만 기다리고 있는 내 모습이 너무 초라하게 느껴져서 눈물이 나려 해…  \n우리 이대로 멀어지게 되는 건 아닐까 걱정돼…  \n\n어떻게 하면 우리 다시 예전처럼 돌아갈 수 있을까?""")

너랑 싸운 뒤로 하루 종일 마음이 너무 불편해…  
네가 나한테 그렇게 화낼 줄 몰랐어서 더 그런가 봐…  
계속 네 연락만 기다리고 있는 내 모습이 너무 초라하게 느껴져서 눈물이 나려 해…  
우리 이대로 멀어지게 되는 건 아닐까 걱정돼…  

어떻게 하면 우리 다시 예전처럼 돌아갈 수 있을까?


In [ ]:
request_data = {
    "messages": [
        {
            "role": "system",
            "content": system_message_situation
        }
    ],
    "thinking": {
        "effort": "none"
    },
    "topP": 0.8,
    "topK": 0,
    "maxCompletionTokens": 1024,
    "temperature": 0.7,
    "repetitionPenalty": 1.1,
    "seed": 0,
    "includeAiFilters": False
}

In [27]:
class LangChainExecutor:
    """LangChain 실행 클래스"""
    def __init__(self, api_key: str):
        self.chat = ChatClovaX(
            model="HCX-007",
            temperature=0.7,
            max_completion_tokens=1024,
            top_p=0.8,
            top_k=0,
            repetition_penalty=1.1,
            seed=0,
            api_key=api_key,
            reasoning = "none",
        )

    def execute(self, messages: List[tuple]) -> Dict[str, Any]:
        start_time = time.time()
        memory_before = psutil.Process().memory_info().rss / 1024 / 1024  # MB
        
        try:
            response = self.chat.invoke(messages)
            first_token_time = time.time() - start_time  # 첫 토큰까지의 시간
            
            response_text = response.content
            # LangChain에서는 토큰 수를 직접 제공하지 않으므로 대략적으로 계산
            total_tokens = len(response_text.split()) * 1.3  # 대략적인 토큰 수 추정
            
        except Exception as e:
            print(f"LangChain request failed: {e}")
            return None
        
        end_time = time.time()
        memory_after = psutil.Process().memory_info().rss / 1024 / 1024  # MB
        
        total_time = end_time - start_time
        tps = total_tokens / total_time if total_time > 0 else 0
        
        return {
            'response_text': response_text,
            'total_time': total_time,
            'ttft': first_token_time,
            'total_tokens': total_tokens,
            'tps': tps,
            'memory_usage': memory_after - memory_before
        }

In [25]:
langchain_executor = LangChainExecutor(
    api_key=api_key,
)

/Users/hongbikim/anaconda3/envs/natna/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3699: UserWarning: WARNING! max_completion_tokens is not default parameter.
                max_completion_tokens was transferred to model_kwargs.
                Please confirm that max_completion_tokens is what you intended.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
langchain_executor.execute(system_message_situation)

LangChain request failed: Completions.create() got an unexpected keyword argument 'reasoning'
